In [1]:
from google.colab import drive
import os
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! ls '/content/drive/'

MyDrive


In [5]:
os.chdir('/content/drive/MyDrive/Bundesliga')

In [6]:
! pwd

/content/drive/MyDrive/Bundesliga


In [7]:
# ! unzip dfl-bundesliga-data-shootout.zip

In [10]:
df_train = pd.read_csv('/content/drive/MyDrive/Bundesliga/train.csv')

In [18]:
df_events = df_train[~df_train['event_attributes'].isna()]
df_events

,video_id,time,event,event_attributes
1,1606b0e6_0,201.150,challenge,['ball_action_forced']
4,1606b0e6_0,210.870,challenge,['opponent_dispossessed']
7,1606b0e6_0,219.230,throwin,['pass']
10,1606b0e6_0,224.430,play,"['pass', 'openplay']"
13,1606b0e6_0,229.390,play,"['pass', 'openplay']"
...,...,...,...,...
11206,ecf251d4_0,3041.347,play,"['pass', 'openplay']"
11209,ecf251d4_0,3050.347,play,"['pass', 'openplay']"
11210,ecf251d4_0,3053.067,play,"['pass', 'openplay']"
11213,ecf251d4_0,3056.587,challenge,['opponent_dispossessed']


In [19]:
df_pass = df_train[df_train['event_attributes'].str[:6] == "['pass"]
df_pass

,video_id,time,event,event_attributes
7,1606b0e6_0,219.230,throwin,['pass']
10,1606b0e6_0,224.430,play,"['pass', 'openplay']"
13,1606b0e6_0,229.390,play,"['pass', 'openplay']"
16,1606b0e6_0,236.710,play,"['pass', 'openplay']"
17,1606b0e6_0,239.350,play,"['pass', 'openplay']"
...,...,...,...,...
11203,ecf251d4_0,3032.587,play,"['pass', 'openplay']"
11206,ecf251d4_0,3041.347,play,"['pass', 'openplay']"
11209,ecf251d4_0,3050.347,play,"['pass', 'openplay']"
11210,ecf251d4_0,3053.067,play,"['pass', 'openplay']"


In [24]:
df_events['event_attributes'].value_counts()

['pass', 'openplay']                  3337
['ball_action_forced']                 239
['pass']                               154
['opponent_dispossessed']              138
['pass', 'freekick']                   127
['fouled']                             111
['cross', 'openplay']                   80
['challenge_during_ball_transfer']      53
['possession_retained']                 44
['opponent_rounded']                    39
['cross', 'corner']                     33
['cross']                               18
['cross', 'freekick']                    5
['pass', 'corner']                       4
Name: event_attributes, dtype: int64

In [31]:
print(f"Accuracy if classifying every class as {df_events['event_attributes'].describe()['top']}: "
      f"{df_events['event_attributes'].describe()['freq'] / df_events['event_attributes'].describe()['count']:.2%}")

Accuracy if classifying every class as ['pass', 'openplay']: 76.15%


In [32]:
print(f'Accuracy if classifying every class as a pass (in general) {df_pass.shape[0] / df_events.shape[0]:.2%}')

Accuracy if classifying every class as a pass (in general) 82.66%
